In [2]:
import copy
import numpy as np
import ipywidgets as widgets

class Individuo:
    def __init__(self, alelos, longitud_gen, cromosoma):
        self._alelos = alelos
        self._longitud_gen = longitud_gen
        self._cromosoma = cromosoma
        self._fitness = 0

In [3]:
class AG:
    def __init__(self, cantidad_individuos, alelos, tamano_gen, generaciones, p, problema):
        self._cantidad_individuos = cantidad_individuos
        self._alelos = alelos
        self._tamano_gen = tamano_gen
        self._generaciones = generaciones
        self._p = p
        self._problema = problema
        self._individuos = np.array([])

    def run(self):
        self.crearIndividuos()
        self._mejor_historico = self._individuos[0]
        generacion = 1
        while generacion <= self._generaciones:
            self.evaluaIndividuos()
            hijos = np.array([])
            while len(hijos) < len(self._individuos):
                padre1 = self.ruleta()
                padre2 = self.ruleta()
                while padre1 == padre2:
                    padre2 = self.ruleta()
                h1, h2 = self.cruza(self._individuos[padre1], self._individuos[padre2])
                hijos = np.append(hijos, [h1])
                hijos = np.append(hijos, [h2])
            self.mutacion(hijos)
            self._individuos = np.copy(hijos)
            self._individuos[np.random.randint(len(self._individuos))] = copy.deepcopy(self._mejor_historico)
            if generacion % 500 == 0:
                print("Generación: ", generacion, 'Mejor Histórico: ', self._mejor_historico._cromosoma, self._mejor_historico._fitness)
            generacion += 1

    def crearIndividuos(self):
        for i in range(self._cantidad_individuos):
            cromosoma = np.random.randint(2, size = self._alelos)
            individuo = Individuo(self._alelos, self._tamano_gen, cromosoma)
            self._individuos = np.append(self._individuos, [individuo])

    def evaluaIndividuos(self):
        for i in self._individuos:
            i._fitness = self._problema.f(self._tamano_gen, i._cromosoma)
            if i._fitness > self._mejor_historico._fitness:
                self._mejor_historico = copy.deepcopy(i)

    def ruleta(self):
        f_sum = np.sum([i._fitness for i in self._individuos])
        if f_sum < 0:
            f_sum *= -1
        if f_sum == 0:
            return np.random.randint(len(self._individuos))
        else:
            r = np.random.randint(f_sum + 1)
            k = 0
            F = self._individuos[k]._fitness
            if F < 0:
                F *= -1
            while F < r and k < len(self._individuos):
                k += 1
                if self._individuos[k]._fitness < 0:
                    F += self._individuos[k]._fitness * -1
                else:
                    F += self._individuos[k]._fitness
            return k

    def cruza(self, i1, i2):
        h1 = copy.deepcopy(i1)
        h2 = copy.deepcopy(i2)

        s = self._alelos - 1
        punto_cruza = np.random.randint(s) + 1
        h1._cromosoma[punto_cruza:], h2._cromosoma[punto_cruza:] = h2._cromosoma[punto_cruza:], h1._cromosoma[punto_cruza:]
        return h1, h2

    def mutacion(self, hijos):
        for h in hijos:
            for bit in range(len(h._cromosoma)):
                if np.random.rand() < self._p:
                    h._cromosoma[bit] = int(not h._cromosoma[bit])

In [35]:
class Horario:
    def __init__(self, secciones, preferencias):
        self._secciones = secciones
        self._preferencias = preferencias

    def f(self, tamano_gen, cromosoma):
        f = 0
        horario = []
        for i in range(len(cromosoma)):
            if cromosoma[i]:
                horario.append(self._secciones[i])
        if self.cumple_restricciones(horario):
          f = self.calcula_puntuacion(horario) + 5
        else:
          f = 0
        return f

    def cumple_restricciones(self, horario):
        # Comprobar si el horario cumple con las restricciones
        # (no se lleva la misma materia dos veces, no se solapan horarios, etc.)
        materias = [seccion.materia for seccion in horario]
        if len(materias) != len(set(materias)):
            return False
        for i in range(len(horario)):
            for j in range(i + 1, len(horario)):
                if horario[i].dias == horario[j].dias and horario[i].horas == horario[j].horas:
                    return False
        return True

    def calcula_puntuacion(self, horario):
        # Calcular la puntuación del horario con base en las preferencias del alumno
        puntuacion = 0
        for seccion in horario:
            if seccion.profesor in self._preferencias['profesor']:
                puntuacion += 1
            if seccion.dias in self._preferencias['dias']:
                puntuacion += 1
            if seccion.horas in self._preferencias['horas']:
                puntuacion += 1
        return puntuacion

    def mostrar_horario(self, cromosoma):
        horario_asignado = []
        for i in range(len(cromosoma)):
            if cromosoma[i] == 1:
                horario_asignado.append(self._secciones[i])
        print('\nHorario: ')
        for seccion in horario_asignado:
            print(f"{seccion.materia} - {seccion.seccion} - {seccion.dias} - {seccion.horas} - {seccion.profesor}")

# Clase para las secciones
class Seccion:
    def __init__(self, materia, seccion, dias, horas, profesor):
        self.materia = materia
        self.seccion = seccion
        self.dias = dias
        self.horas = horas
        self.profesor = profesor

In [ ]:
def main():
    # Secciones
    secciones = [
        Seccion(materia="Español", seccion="D02", dias='L-I', horas='1500-1655', profesor='JULIO GABRIEL'),
        Seccion(materia="Español", seccion="D03", dias='L-I', horas='1500-1655', profesor='ANA MARÍA'),
        Seccion(materia="Español", seccion="D04", dias='M-J', horas='0900-1055', profesor='CARLOS ALBERTO'),
        Seccion(materia="Español", seccion="D05", dias='S', horas='0900-1255', profesor='ANA MARÍA'),
        Seccion(materia="Español", seccion="D06", dias='M-J', horas='0900-1055', profesor='FERNANDO ANDRÉ'),
        Seccion(materia="Matemáticas", seccion="D02", dias='V', horas='1100-1455', profesor='CARLOS MARTÍNEZ'),
        Seccion(materia="Matemáticas", seccion="D03", dias='L-I', horas='1100-1255', profesor='ANA GARCÍA'),
        Seccion(materia="Matemáticas", seccion="D04", dias='M-J', horas='1300-1455', profesor='MARTÍN SÁNCHEZ'),
        Seccion(materia="Matemáticas", seccion="D05", dias='M-J', horas='0900-1055', profesor='SOFÍA RAMÍREZ'),
        Seccion(materia="Matemáticas", seccion="D04", dias='L-I', horas='1500-1655', profesor='MARTÍN SÁNCHEZ'),
        Seccion(materia="Historia", seccion="D02", dias='M-J', horas='1100-1255', profesor='ESTEBAN MORALES'),
        Seccion(materia="Historia", seccion="D03", dias='L-I', horas='0700-0855', profesor='KHVICHA KVARATSKHELIA'),
        Seccion(materia="Historia", seccion="D04", dias='M-J', horas='1300-1455', profesor='LIONEL RONALDO'),
        Seccion(materia="Historia", seccion="D05", dias='M-J', horas='0900-1055', profesor='JUAN RIQUELME'),
        Seccion(materia="Historia", seccion="D03", dias='L-I', horas='1100-1255', profesor='KHVICHA KVARATSKHELIA'),
        Seccion(materia="Inglés", seccion="D02", dias='M-J', horas='0900-1055', profesor='DAVID WILLIAMS'),
        Seccion(materia="Inglés", seccion="D03", dias='V', horas='1100-1455', profesor='SARAH ANDERSON'),
        Seccion(materia="Inglés", seccion="D04", dias='L-I', horas='1300-1455', profesor='MICHAEL JOHNSON'),
        Seccion(materia="Inglés", seccion="D05", dias='L-I', horas='1500-1655', profesor='EMMA WHITE'),
        Seccion(materia="Inglés", seccion="D06", dias='M-J', horas='1600-1855', profesor='OLIVIA BROWN'),
    ]

    # Preferecnias del alumno
    alumno_preferencias = {
        "profesor": ["KHVICHA KVARATSKHELIA", "ANA MARÍA"], # Preferencia por el profesor
        "dias": ['L-I', 'M-J'],     # Días preferidos
        "horas": ['1100-1255', '0900-1055']  # Horas preferidas
    }

    alelos = len(secciones)
    h = Horario(secciones, alumno_preferencias)
    individuos = 36
    tamano_gen = 1  # cada bit representa una moneda tomada
    generaciones = 4500
    factor_mutacion = 0.01
    ag = AG(individuos, alelos, tamano_gen, generaciones, factor_mutacion, h)
    ag.run()

    # Obtener la mejor solución encontrada
    mejor_solucion = ag._mejor_historico

    # Mostrar el horario asignado por la mejor solución
    h.mostrar_horario(mejor_solucion._cromosoma)

if __name__ == "__main__":
  for i in range(3):
    print(f'\nEjecución {i + 1}')
    main()